# Parte dos. Predecir sobre el dataset dev

In [1]:
import pandas as pd
import sys
import os

df = pd.read_csv('../data/TA1C_dataset_detection_dev.csv')

df = df.drop(['Tweet Date', 'Media Name', 'Media Origin'], axis=1)

In [2]:
# Obtener la ruta absoluta de la carpeta 'src' para que sea accesible desde el notebook
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from preprocessing import normalize_text

df['normalized_text'] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="text_cleaning")).apply(lambda x: normalize_text(x, mode="lemmatization"))
# Si cada celda es una lista de tokens, conviértela a string
df['normalized_text'] = df['normalized_text'].apply(lambda tokens: ' '.join(tokens) if isinstance(tokens, list) else tokens)

X = df['normalized_text']


In [4]:
import joblib

mlp_model = joblib.load('../model/mlp_model.joblib')
vectorizer = joblib.load('../model/vectorizer.joblib')
svd = joblib.load('../model/svd.joblib')

X_vect = vectorizer.transform(X)
X_reduced = svd.transform(X_vect)
y_pred = mlp_model.predict(X_reduced)

df['Tag Value'] = y_pred

label_mapping = {
    1 : 'Clickbait',
    0 : 'No'
}

df['Tag Value'] = df['Tag Value'].map(label_mapping)

df = df.drop(['Teaser Text', 'normalized_text'], axis=1)

df.to_csv('../data_dev/detection.csv', index=False)